In [ ]:
#bd
def getHistoricalAPI(token,interval= 'ONE_MINUTE'):
    to_date= datetime.now()
    from_date = to_date - timedelta(days=5)
    from_date_format = from_date.strftime("%Y-%m-%d %H:%M")
    to_date_format = to_date.strftime("%Y-%m-%d %H:%M")
    try:
        historicParam={
        "exchange": "NSE",
        "symboltoken": token,
        "interval": interval,
        "fromdate": from_date_format, 
        "todate": to_date_format
        }
        candel_json  = credentials.SMART_API_OBJ.getCandleData(historicParam)
        return calculate_inidcator(candel_json)
    except Exception as e:
        print("Historic Api failed: {}".format(e.message))
        
def calculate_bd_inidcator(res_json):
    columns = ['timestamp','O','H','L','C','V']
    df = pd.DataFrame(res_json['data'], columns=columns)
    df['timestamp'] = pd.to_datetime(df['timestamp'],format = '%Y-%m-%dT%H:%M:%S')
#     df['EMA'] = EMA(df.C, timeperiod=20)
    df['upperband'], df['middleband'], df['lowerband'] = BBANDS(df.C, timeperiod=20, nbdevup=20, nbdevdn=2, matype=0)
    df['RSI'] = RSI(df.C, timeperiod=14)
    df['ATR'] = ATR(df.H, df.L, df.C, timeperiod=20)
    df['CROSS_UP'] = df['CROSS_DOWN'] =df['RSI_UP'] = 0
    df = df.round(decimals=2)
    
    for i in range(20,len(df)):
        if ( df['upperband'][i-1]-df['C'][i-1] in range(-0.1,0.1)) and ( df['upperband'][i]-df['C'][i] in range(-0.1,0.1)):
            df['CROSS_UP'][i] = 1
        if ( df['lowerband'][i-1]-df['C'][i-1] in range(-0.1,0.1)) and ( df['lowerband'][i]-df['C'][i] in range(-0.1,0.1)):
            df['CROSS_DOWN'][i] = 1
        if df['RSI'][i] > 50 : 
            df['RSI_UP'][i] = 1 
   
    print(df.tail(10))
    return df

